## Import

In [10]:
import cv2
import numpy as np

## Initialization

In [9]:
middlebury = [
    ('./middlebury-dataset/00/l.png', './middlebury-dataset/00/r.png'),
    ('./middlebury-dataset/01/l.png', './middlebury-dataset/01/r.png'),
    ('./middlebury-dataset/02/l.png', './middlebury-dataset/02/r.png'),
    ('./middlebury-dataset/03/l.png', './middlebury-dataset/03/r.png'),
    ('./middlebury-dataset/04/l.png', './middlebury-dataset/04/r.png'),
    ('./middlebury-dataset/05/l.png', './middlebury-dataset/05/r.png'),
    ('./middlebury-dataset/06/l.png', './middlebury-dataset/06/r.png'),
    ('./middlebury-dataset/07/l.png', './middlebury-dataset/07/r.png'),
    ('./middlebury-dataset/08/l.png', './middlebury-dataset/08/r.png'),
    ('./middlebury-dataset/09/l.png', './middlebury-dataset/09/r.png')
    ]

In [22]:
real_world = [
    ('real-world-dataset/01/l.jpg', 'real-world-dataset/01/r.jpg'),
    ('real-world-dataset/04/l.jpg', 'real-world-dataset/04/r.jpg'),
    ('real-world-dataset/06/l.jpg', 'real-world-dataset/06/r.jpg'),
    ('real-world-dataset/07/l.jpg', 'real-world-dataset/07/r.jpg')
]

## Code

In [12]:
def load_image_pairs(image_pairs):
    loaded_pairs = []

    for left_path, right_path in image_pairs:
        left_image = cv2.imread(left_path, cv2.IMREAD_GRAYSCALE)
        right_image = cv2.imread(right_path, cv2.IMREAD_GRAYSCALE)

        if left_image is None or right_image is None:
            continue
        
        loaded_pairs.append((left_image, right_image))

    return loaded_pairs

In [25]:
# ORB + BFMatcher -> SIFT + RANSAC
def find_keypoints_and_matches(left_image, right_image):
    # Using SIFT detector to find keypoints and descriptors.
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(left_image, None)
    kp2, des2 = sift.detectAndCompute(right_image, None)

    # Using FLANN-based matching.
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Filtering the matches.
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    return kp1, kp2, good_matches

In [14]:
def compute_fundamental_matrix(kp1, kp2, matches):
    if len(matches) < 8:
        return None, None

    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

    # Calculating the Fundamental Matrix.
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

    return F, mask

## Main

In [26]:
# Middlebury dataset
middlebury_output = 'middlebury.txt'
middlebury_pairs = load_image_pairs(middlebury)

with open(middlebury_output, "w") as file:
    for idx, (left_image, right_image) in enumerate(middlebury_pairs):
        kp1, kp2, matches = find_keypoints_and_matches(left_image, right_image)
        F, mask = compute_fundamental_matrix(kp1, kp2, matches)

        if F is not None:
            print(f"Image Index {idx}:")
            print(F)
            file.write(f"Image Index {idx}:\n")
            file.write(f"{F}\n")
        else:
            print(f"Image Index {idx}: Not enough matches to compute Fundamental Matrix")

Image Index 0:
[[-2.37200370e-09 -2.94928719e-06  5.09925474e-03]
 [ 2.82272949e-06  1.00826348e-06  1.57589540e-01]
 [-4.97866084e-03 -1.58988791e-01  1.00000000e+00]]
Image Index 1:
[[-5.13598427e-08 -4.17883050e-05  1.04751045e-02]
 [ 3.97685481e-05  1.92095156e-06 -1.87642052e-01]
 [-9.80677753e-03  1.84395207e-01  1.00000000e+00]]
Image Index 2:
[[ 4.11307067e-08 -2.63575996e-05  3.80438294e-02]
 [ 2.63667572e-05 -3.01502088e-06  1.33715804e-01]
 [-3.78774670e-02 -1.30159924e-01  1.00000000e+00]]
Image Index 3:
[[ 2.84695609e-07  7.29223946e-05 -1.36077582e-01]
 [-5.92894442e-05  5.25557707e-06 -2.41491907e+00]
 [ 1.19803906e-01  2.39937298e+00  1.00000000e+00]]
Image Index 4:
[[-5.66839288e-08 -1.37677674e-04  3.33394744e-02]
 [ 1.38159158e-04 -1.78767733e-06 -3.74052672e-02]
 [-3.33874187e-02  3.36464189e-02  1.00000000e+00]]
Image Index 5:
[[-8.65630573e-08 -1.33249206e-04  9.37540983e-02]
 [ 1.33039549e-04 -9.57413277e-07  5.36865569e-02]
 [-9.35817003e-02 -5.43998296e-02  1.0

In [27]:
# Real world dataset
real_output = 'real_world.txt'
real_pairs = load_image_pairs(real_world)

with open(real_output, "w") as file:
    for idx, (left_image, right_image) in enumerate(real_pairs):
        kp1, kp2, matches = find_keypoints_and_matches(left_image, right_image)
        F, mask = compute_fundamental_matrix(kp1, kp2, matches)

        if F is not None:
            print(f"Image Index {idx}:")
            print(F)
            file.write(f"Image Index {idx}:\n")
            file.write(f"{F}\n")
        else:
            print(f"Image Index {idx}: Not enough matches to compute Fundamental Matrix")

Image Index 0:
[[ 1.50828804e-08 -6.43160878e-07  1.59408283e-03]
 [ 8.05516819e-07  5.93905824e-09  5.91664031e-03]
 [-1.88002353e-03 -6.43620986e-03  1.00000000e+00]]
Image Index 1:
[[-8.23790027e-09 -1.51370012e-07 -1.28390778e-03]
 [ 5.69367404e-08  7.36293100e-08 -1.36621256e-02]
 [ 1.16252283e-03  1.35850534e-02  1.00000000e+00]]
Image Index 2:
[[ 3.89965065e-07 -1.57647026e-05  4.39892041e-02]
 [ 1.35514473e-05 -7.73219349e-07 -1.11558530e-01]
 [-4.07202566e-02  1.13795279e-01  1.00000000e+00]]
Image Index 3:
[[ 4.61548423e-09  4.07757641e-07 -8.73019501e-04]
 [-3.13436932e-07 -1.40416651e-07  1.94810481e-02]
 [ 4.36728690e-04 -1.92834504e-02  1.00000000e+00]]
Image Index 4:
[[-2.10434638e-08 -1.52903188e-06  2.81066362e-03]
 [ 1.47854529e-06  8.34527524e-08  1.93296539e-02]
 [-2.90509077e-03 -1.94636963e-02  1.00000000e+00]]
Image Index 5:
[[ 1.67300647e-08 -1.24015504e-06  2.50651678e-03]
 [ 1.23781270e-06 -7.89665304e-08 -1.26722321e-02]
 [-2.74941041e-03  1.27372269e-02  1.0